In [1]:
!pip install wget

In [2]:
!pip install wandb -qqq
import wandb
wandb.login()

wandb: Currently logged in as: mak109 (use `wandb login --relogin` to force relogin)


True

In [3]:
from wandb.keras import WandbCallback

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import random
import wget
import os
from zipfile import ZipFile
from PIL import Image
plt.rcParams["figure.figsize"] = (20,10)

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential,regularizers,optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
random.seed(123)

In [6]:
import wget
import os
from zipfile import ZipFile
url='https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
filename = os.path.basename(url)

if not os.path.exists(filename) and not os.path.exists("inaturalist_12K"):
  filename = wget.download(url)
  with ZipFile(filename, 'r') as zip:
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')
  os.remove(filename)

In [67]:
image_size = (256,256)
num_layers = 5
num_classes = 10

In [80]:
tf.debugging.set_log_device_placement(True)

In [68]:
def CNN(config):
    model = Sequential([
        layers.Input((image_size[0],image_size[1],3)),
        layers.Rescaling(1./255)
        ])
    
    for l in range(num_layers):
        model.add(layers.Conv2D(filters=config["filters_list"][l],kernel_size=(config["kernel_sizes"][l][0],config["kernel_sizes"][l][1]),
                        activation=config["activation"],padding="same",kernel_regularizer=regularizers.l2(config["weight_decay"])))
        if config["batch_normalization"] == 'True':
            model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(config["dense_layer_size"],activation=config["activation"],kernel_regularizer=regularizers.l2(config["weight_decay"])))
    model.add(layers.Dropout(config["dropout"]))

    model.add(layers.Dense(num_classes,activation="softmax"))
    return model



In [82]:
#Training goes here

def train():
    #default configuration
    config_ = {
    "kernel_sizes" : [(3,3),(3,3),(3,3),(3,3),(3,3)],
    "activation" : 'relu',
    "learning_rate": 1e-3,
    "filters_list" : [32,32,64,64,32],
    "dense_layer_size" : 128,
    "batch_normalization": "True",
    "data_augment": "False",
    "weight_decay":0.0005,
    "dropout":0.2,
    "batch_size":64,
    "epochs":3
    }
    wandb.init(config=config_)
    config = wandb.config
    
    #Setting run name for better readability
    wandb.run.name = "nd_"+str(config["dense_layer_size"])+"bs_"+str(config["batch_size"])+"ac_"+str(config["activation"])
    #Some data preprocessing and train,val splitting
    
    val_generator = ImageDataGenerator(dtype=tf.float32,validation_split=0.1,data_format='channels_last').flow_from_directory(
        'inaturalist_12K/train',
        target_size = image_size,
        batch_size = config['batch_size'],
        color_mode = 'rgb',
        class_mode = 'sparse',
        shuffle=True,
        subset='validation',
        seed=123
    
    )
    #Data Augmentation
    if config["data_augment"] == 'True':
        data_generator = ImageDataGenerator(
        rotation_range=50, #random rotation between -50(clockwise) to 50(anti-clockwise) degree
        brightness_range=(0.2,0.8), 
        zoom_range=0.3, #zoom in range from [0.7,1.3]
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.1, #Horizontal Shifting as a ratio of width
        height_shift_range=0.2,#Vertical Shifting as a ratio of height
        data_format='channels_last',
        validation_split=0.1,
        dtype=tf.float32
        )
    else:
        data_generator = ImageDataGenerator(
            data_format='channels_last',
            validation_split=0.1,
            dtype=tf.float32
        )
    #Train set creation after conditional augmentation
    train_generator = data_generator.flow_from_directory(
    'inaturalist_12K/train',
    target_size = image_size,
    batch_size = config['batch_size'],
    color_mode = 'rgb',
    class_mode = 'sparse',
    shuffle=True,
    subset='training',
    seed=123
    )
    #Building Model based on config 
    model = CNN(config)
    
    #Compiling model 
    model.compile(
    optimizer=optimizers.Adam(learning_rate=config["learning_rate"]),
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
    )
    
    #Fitting Model
    model.fit(train_generator,
        validation_data=val_generator,
        epochs=config["epochs"],
        verbose=1,
        callbacks = [WandbCallback()]
        )
    #Saving model into run directory for future testing
    model.save(os.path.join(wandb.run.dir,"model.h5"))
    wandb.finish()

In [85]:
#Sweep configuration for runs
sweep_config = {
  "name" : "best-sweep",
  "method" : "bayes",
    "early_terminate":{
        "type":"hyperband",
        "max_iter":27,
        "s":2
    },
  "metric" : {
      "name" : "val_accuracy",
      "goal" : "maximize"
  },
  "parameters" : {
    "epochs" : {
      "values" : [10,20,30]
    },
    "learning_rate" :{
      "values" : [1e-3,1e-4]
    },
    "kernel_sizes":{
        "values" : [[(3,3),(3,3),(3,3),(3,3),(3,3)],
                    [(3,3),(3,3),(5,5),(7,7),(7,7)],
                    [(11,11),(11,11),(7,7),(5,5),(3,3)],
                    [(3,3),(5,5),(7,7),(9,9),(11,11)]]
    },
    "filters_list":{
        "values" : [[32,32,32,32,32],[128,64,64,32,32],[32,64,64,128,128],[32,64,128,256,512]]
    },
    "weight_decay":{
      "values": [0,0.0005,0.005]  
    },
    "data_augment":{
        "values": ["True","False"]
    },
    "batch_size":{
        "values":[32,64]
    },
    "activation":{
        "values": ["relu","elu","swish"]
    },
      "dropout":{
          "values":[0.0,0.2,0.3]
      },
      "dense_layer_size":{
          "values":[64,128,256,512]
      },
      "batch_normalization":{
          "values":["True","False"]
      }
  }
}

In [86]:
sweep_id=wandb.sweep(sweep_config,entity="dlstack",project="CS6910-ASSIGNMENT-2")

Create sweep with ID: hhts08vf
Sweep URL: https://wandb.ai/dlstack/CS6910-ASSIGNMENT-2/sweeps/hhts08vf


In [ ]:
wandb.agent(sweep_id, function=train, count=1)

wandb: Agent Starting Run: tt6xober with config:
wandb: 	activation: relu
wandb: 	batch_normalization: True
wandb: 	batch_size: 32
wandb: 	data_augment: True
wandb: 	dense_layer_size: 256
wandb: 	dropout: 0.4
wandb: 	epochs: 20
wandb: 	filters_list: [128, 64, 64, 32, 32]
wandb: 	kernel_sizes: [[11, 11], [11, 11], [7, 7], [5, 5], [3, 3]]
wandb: 	learning_rate: 0.0001
wandb: 	weight_decay: 0


Found 999 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Epoch 1/20


/opt/homebrew/Caskroom/miniforge/base/envs/mak/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2022-03-25 11:54:20.541422: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


282/282 [==============================] - ETA: 0s - loss: 2.6761 - accuracy: 0.1427

2022-03-25 12:17:56.442517: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


282/282 [==============================] - 1446s 5s/step - loss: 2.6761 - accuracy: 0.1427 - val_loss: 2.4446 - val_accuracy: 0.1261 - _timestamp: 1648190905.0000 - _runtime: 1451.0000
Epoch 2/20
282/282 [==============================] - 1430s 5s/step - loss: 2.3010 - accuracy: 0.1782 - val_loss: 2.2102 - val_accuracy: 0.2012 - _timestamp: 1648192335.0000 - _runtime: 2881.0000
Epoch 3/20
282/282 [==============================] - 1396s 5s/step - loss: 2.2144 - accuracy: 0.1992 - val_loss: 2.5335 - val_accuracy: 0.1592 - _timestamp: 1648193732.0000 - _runtime: 4278.0000
Epoch 4/20
282/282 [==============================] - 1411s 5s/step - loss: 2.1789 - accuracy: 0.2118 - val_loss: 2.3082 - val_accuracy: 0.1582 - _timestamp: 1648195142.0000 - _runtime: 5688.0000
Epoch 5/20
 96/282 [=========>....................] - ETA: 16:00 - loss: 2.1557 - accuracy: 0.2106

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13GFamilyCommandBuffer: 0x28a6b8430>
    label = <none> 
    device = <AGXG13GDevice: 0x156f19a00>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x157901c00>
        label = <none> 
        device = <AGXG13GDevice: 0x156f19a00>
            name = Apple M1 
    retainedReferences = 1


282/282 [==============================] - 1465s 5s/step - loss: 2.1532 - accuracy: 0.2118 - val_loss: 2.3620 - val_accuracy: 0.1622 - _timestamp: 1648196608.0000 - _runtime: 7154.0000
Epoch 6/20
282/282 [==============================] - 1428s 5s/step - loss: 2.1368 - accuracy: 0.2237 - val_loss: 2.3004 - val_accuracy: 0.1632 - _timestamp: 1648198036.0000 - _runtime: 8582.0000
Epoch 7/20
282/282 [==============================] - 1434s 5s/step - loss: 2.1078 - accuracy: 0.2414 - val_loss: 2.1464 - val_accuracy: 0.2322 - _timestamp: 1648199469.0000 - _runtime: 10015.0000
Epoch 8/20
 28/282 [=>............................] - ETA: 20:28 - loss: 2.1145 - accuracy: 0.2400